In [13]:
# Importaciones necesarias para construir y entrenar redes neuronales con TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras import layers, models  # Crear modelos y capas
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Aumentación de datos
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping  # Callbacks para mejorar el entrenamiento
from tensorflow.keras.optimizers import Adam  # Optimizador Adam
import numpy as np  # Operaciones matemáticas y manejo de datos
import matplotlib.pyplot as plt  # Visualización de gráficos
from tensorflow.keras.datasets import cifar10  # Conjunto de datos CIFAR-10

# **1. Cargar y preprocesar los datos de CIFAR-10**
# CIFAR-10 contiene imágenes (32x32) divididas en 10 clases
# Aumentación de datos para enriquecer el conjunto de entrenamiento
data_augmentation = ImageDataGenerator(
    rotation_range=15,        # Rotación aleatoria de hasta 15 grados
    width_shift_range=0.1,    # Desplazamiento horizontal aleatorio del 10% del ancho
    height_shift_range=0.1,   # Desplazamiento vertical aleatorio del 10% de la altura
    horizontal_flip=True      # Volteo horizontal aleatorio
)

# Cargar los datos de entrenamiento y prueba
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir las etiquetas numéricas a formato one-hot encoding (necesario para clasificación multiclase)
y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)

# **2. Normalización de datos**
# Normaliza las imágenes con respecto al promedio y la desviación estándar (simulando la preparación de datos de ImageNet)
mean = np.mean(x_train, axis=(0,1,2)) / 255.0  # Promedio de los valores de píxeles
std = np.std(x_train, axis=(0,1,2)) / 255.0    # Desviación estándar de los valores de píxeles
x_train = (x_train / 255.0 - mean) / std       # Normaliza el conjunto de entrenamiento
x_test = (x_test / 255.0 - mean) / std         # Normaliza el conjunto de prueba

# Define la forma de entrada (32x32 imágenes RGB)
input_shape = (32, 32, 3)

# **3. Definir el modelo AlexNet desde cero**
def alexnet():
    # Se utiliza un modelo secuencial donde las capas se apilan linealmente
    model = models.Sequential([
        # Primera capa convolucional con 96 filtros, kernel 3x3 y activación ReLU
        layers.Conv2D(96, (3, 3), strides=1, activation='relu', input_shape=input_shape, padding='same'),
        # MaxPooling reduce las dimensiones espaciales
        layers.MaxPooling2D((3, 3), strides=2, padding='same'),

        # Segunda capa convolucional con kernel 5x5 y 256 filtros
        layers.Conv2D(256, (5, 5), activation='relu', padding='same'),
        layers.MaxPooling2D((3, 3), strides=2, padding='same'),

        # Tres capas convolucionales consecutivas
        layers.Conv2D(384, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(384, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((3, 3), strides=2, padding='same'),

        # Aplanar la salida de las capas convolucionales para conectarlas con las densas
        layers.Flatten(),

        # Primera capa densa con 4096 neuronas
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),  # Dropout para evitar sobreajuste

        # Segunda capa densa con 4096 neuronas
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),

        # Capa de salida para clasificar en 10 clases (softmax para clasificación multiclase)
        layers.Dense(10, activation='softmax')
    ])
    return model

# **4. Compilar el modelo**
model = alexnet()
model.compile(optimizer=Adam(learning_rate=0.001),  # Optimizador Adam con tasa de aprendizaje inicial 0.001
              loss='categorical_crossentropy',      # Función de pérdida para clasificación multiclase
              metrics=['accuracy'])                # Métrica de precisión para el monitoreo

# **5. Configurar callbacks para el entrenamiento**
callbacks = [
    # Reduce la tasa de aprendizaje si no mejora la pérdida en 3 épocas consecutivas
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    # Detiene el entrenamiento si no hay mejora en 7 épocas consecutivas
    EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)
]

# **6. Entrenar el modelo con aumentación de datos**
history = model.fit(
    data_augmentation.flow(x_train, y_train, batch_size=64),  # Genera lotes con aumentación
    epochs=10,                                               # Entrena durante 10 épocas
    validation_data=(x_test, y_test),                        # Datos de validación
    callbacks=callbacks                                      # Aplica los callbacks
)

# **7. Evaluar el modelo**
# Evalúa el modelo final en el conjunto de prueba
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")  # Imprime la precisión final #7 min

# **8. Instalar tf-keras-vis para visualización de activaciones**
!pip install tf-keras-vis


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 58s 63ms/step - accuracy: 0.2765 - loss: 1.9224 - val_accuracy: 0.4423 - val_loss: 1.4787 - learning_rate: 0.0010
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 52ms/step - accuracy: 0.5001 - loss: 1.3797 - val_accuracy: 0.5745 - val_loss: 1.1926 - learning_rate: 0.0010
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.5652 - loss: 1.2280 - val_accuracy: 0.6165 - val_loss: 1.0821 - learning_rate: 0.0010
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 52ms/step - accuracy: 0.5942 - loss: 1.1485 - val_accuracy: 0.6376 - val_loss: 1.0151 - learning_rate: 0.0010
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.6207 - loss: 1.0872 - val_accuracy: 0.6330 - val_loss: 1.1067 - learning_rate: 0.0010
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 50ms/step - accuracy: 0.6424 - loss: 1.0316 - val_accuracy: 0.6556 - val_loss: 0.9893 - learning_rate: 0.0010
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 52ms/step - accuracy: 0.6498 - loss: 1.0008

In [ ]:
from tensorflow.keras.utils import to_categorical  # Conversión de etiquetas a formato one-hot

# **1. Cargar y preprocesar los datos de CIFAR-10**
# CIFAR-10 contiene imágenes de 32x32 píxeles, divididas en 10 clases
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Convertir las etiquetas de formato numérico a one-hot encoding (necesario para clasificación multiclase)
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# **2. Configurar Data Augmentation (Aumentación de datos)**
# Esto enriquece el conjunto de entrenamiento mediante transformaciones aleatorias para mejorar la generalización
datagen = ImageDataGenerator(
    rotation_range=15,         # Rotar imágenes aleatoriamente hasta 15 grados
    width_shift_range=0.1,     # Desplazamiento horizontal aleatorio del 10% del ancho
    height_shift_range=0.1,    # Desplazamiento vertical aleatorio del 10% de la altura
    horizontal_flip=True       # Volteo horizontal aleatorio
)

# Ajustar el generador de datos al conjunto de entrenamiento
datagen.fit(X_train)

# **3. Descargar el modelo AlexNet preentrenado**
# Nota: TensorFlow no incluye AlexNet por defecto, por lo que se utiliza VGG16 como alternativa
alexnet_tf = tf.keras.applications.VGG16(
    weights="imagenet",         # Usar pesos preentrenados en el conjunto de datos ImageNet
    include_top=False,          # Excluir las capas densas originales (solo usar la parte convolucional)
    input_shape=(224, 224, 3)   # Cambiar la entrada para que coincida con ImageNet (224x224, 3 canales RGB)
)

# **4. Adaptar el modelo AlexNet a CIFAR-10**
# Crear un modelo secuencial personalizado que utilice las capas convolucionales preentrenadas
model_tf = models.Sequential([
    layers.Resizing(224, 224),  # Redimensionar imágenes de CIFAR-10 (32x32 -> 224x224)
    alexnet_tf,                 # Modelo convolucional preentrenado VGG16
    layers.Flatten(),           # Aplanar la salida de las convoluciones para conectarla a capas densas
    layers.Dense(4096, activation='relu'),  # Primera capa densa con 4096 neuronas
    layers.Dropout(0.5),        # Dropout para reducir el riesgo de sobreajuste
    layers.Dense(4096, activation='relu'),  # Segunda capa densa con 4096 neuronas
    layers.Dropout(0.5),        # Otro Dropout
    layers.Dense(10, activation='softmax')  # Capa de salida para clasificar 10 clases de CIFAR-10
])

# **5. Congelar capas preentrenadas**
# Las capas convolucionales de VGG16 se congelan (no se entrenan) para usar los pesos preentrenados
for layer in alexnet_tf.layers:
    layer.trainable = False  # Desactivar el entrenamiento de las capas preentrenadas

# **6. Compilar el modelo**
model_tf.compile(
    optimizer=Adam(learning_rate=0.0005),    # Optimizador Adam con tasa de aprendizaje ajustada a 0.0005
    loss="categorical_crossentropy",        # Función de pérdida para clasificación multiclase
    metrics=["accuracy"]                    # Métrica de evaluación: precisión
)

# **7. Entrenar el modelo**
history_tf = model_tf.fit(
    datagen.flow(X_train, y_train, batch_size=64),  # Entrenamiento con datos aumentados (lotes de 64 imágenes)
    validation_data=(X_test, y_test),              # Conjunto de datos de prueba para validación
    epochs=10,                                     # Número de épocas: 10
    verbose=2                                      # Mostrar salida detallada durante el entrenamiento
)

# **8. Evaluar el modelo en el conjunto de prueba**
test_loss_tf, test_acc_tf = model_tf.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy AlexNet (TF Pretrained): {test_acc_tf}")  # Imprime la precisión final


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10
782/782 - 366s - 468ms/step - accuracy: 0.5521 - loss: 2.4002 - val_accuracy: 0.7436 - val_loss: 0.8576
Epoch 2/10
782/782 - 328s - 419ms/step - accuracy: 0.6081 - loss: 1.3132 - val_accuracy: 0.7605 - val_loss: 0.8028
Epoch 3/10
782/782 - 325s - 415ms/step - accuracy: 0.6480 - loss: 1.1766 - val_accuracy: 0.7764 - val_loss: 0.7245
Epoch 4/10
782/782 - 293s - 374ms/step - accuracy: 0.6734 - loss: 1.0891 - val_accuracy: 0.7971 - val_loss: 0.6634
Epoch 5/10
782/782 - 354s - 453ms/step - accuracy: 0.6947 - loss: 1.0027 - val_accuracy: 0.8143 - val_loss: 0.6096
Epoch 6/10
782/782 - 328s - 419ms/step - accuracy: 0.7136 - loss: 0.9355 - val_accuracy: 0.8029 - val_loss: 0.6300
Epoch 7/10
782/782 - 323s - 413ms/step - accuracy: 0.7165 - loss: 0.9137 - val_accuracy: 0.8204 - val_loss: 0.5945
Epoch 8/10
782/782 - 327s - 419ms/step - accuracy: 0.7248 - loss: 0.8946 - val_accuracy: 0.8244 - val_loss: 0.5785
Epoch 9/10
782/782 - 328s - 4